# Text Embedding using Cohere LLM Model and stored in Amazon OpenSearch serverless

---

## Introduction

Embeddings are integral to various natural language processing applications, with their quality crucial for optimal performance. They are commonly used in knowledge bases to represent textual data as dense vectors enabling efficient similarity search and retrieval. Embeddings play a key role in personalization and recommendation systems by representing user preferences, item characteristics, and historical interactions as vectors, allowing calculation of similarities for personalized recommendations based on user behavior and item embeddings.

In this notebook, we demonstrate how to use the Cohere Embed Multilingual V3 LLM (Large Language Model) for creating text embedding that will be stored in Amazon OpenSearch with vector engine support for assisting with the prompt engineering task for more accurate response from LLMs.

## Prepare documents


Before being able to search text based on meaning, not just keywords the questions, the documents must be processed and a stored in a document store index

* Load the documents
* Create a numerical vector representation using Amazon Bedrock Cohere model
* Create an index and the corresponding embeddings in the Amazon Open Search Serverless

## Search Text

When the documents index is prepared, you are ready to search text and relevant documents will be fetched based on the query being asked. Following steps will be executed.

* Create an embedding of the input query
* Compare the query embedding with the embeddings in the index
* Fetch the (top N) relevant document chunks
* Add those chunks as part of the context in the prompt
* Send the prompt to the Cohere Command R+ model under Amazon Bedrock
* Get the contextual answer based on the documents retrieved


---



It's recommended to execute the notebook in SageMaker Studio Notebooks `Python 3.0(Data Science)` Kernel with `ml.t3.medium` instance.

In [ ]:
%load_ext autoreload
%autoreload 2

## Setup

Before running the rest of this notebook, you'll need to run the cells below to ensure necessary libraries are installed.

In [ ]:
!pip install opensearch-py
!pip install requests-aws4auth
!pip install -U boto3
!pip install -U botocore
!pip install -U awscli
!pip install -U datasets
!pip install -U pypdf
!pip install langchain
!pip install langchain-community

Install some python packages we are going to use.

In [ ]:
# External Dependencies:
import warnings

from io import StringIO
import sys
import textwrap
import os
from typing import Optional


import boto3
from botocore.config import Config
import sagemaker
import json
import time
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth, helpers

warnings.filterwarnings('ignore')

In [ ]:
# getting boto3 clients for required AWS services

aoss_client = boto3.client('opensearchserverless')

bedrock_client = boto3.client(
    "bedrock-runtime", 
    "us-east-1", 
    endpoint_url="https://bedrock-runtime.us-east-1.amazonaws.com"
)

session = boto3.session.Session()

region_name = session.region_name

In [ ]:
# Create a SageMaker session
sagemaker_role_arn = sagemaker.get_execution_role()
sagemaker_role_arn

## Load dataset

For this notebook, Let's first download some of the files to build our document store. For this example we will be using public IRS documents from [here](https://www.irs.gov/publications).

In [ ]:
from urllib.request import urlretrieve

os.makedirs("data", exist_ok=True)
files = [
    "https://www.irs.gov/pub/irs-pdf/p1544.pdf",
    "https://www.irs.gov/pub/irs-pdf/p15.pdf",
    "https://www.irs.gov/pub/irs-pdf/p1212.pdf",
]
for url in files:
    file_path = os.path.join("data", url.rpartition("/")[2])
    urlretrieve(url, file_path)

## Data Preparation

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader


loader = PyPDFDirectoryLoader("./data/")

documents = loader.load()
# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=2000,
    chunk_overlap=200,
)
docs = text_splitter.split_documents(documents)

In [ ]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents]) // len(
    documents
)
avg_char_count_pre = avg_doc_length(documents)
avg_char_count_post = avg_doc_length(docs)
print(f"Average length among {len(documents)} documents loaded is {avg_char_count_pre} characters.")
print(f"After the split we have {len(docs)} documents more than the original {len(documents)}.")
print(
    f"Average length among {len(docs)} documents (after split) is {avg_char_count_post} characters."
)

## Generate embedding using Cohere Embed Multilingual V3

Cohere Embed Multilingual V3: An embedding model designed to encode text from various languages into dense vector representations, enabling efficient similarity comparisons and semantic search.


In [ ]:
def create_vector_embedding_with_bedrock(text, bedrock_client):
    payload = {"inputText": f"{text}"}
    body = json.dumps(payload)
    modelId = "amazon.titan-embed-text-v1"
    accept = "application/json"
    contentType = "application/json"

    response = bedrock_client.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    embedding = response_body.get("embedding")
    return {"text": text, "vector_field": embedding}

Lets try embedding first document and check the result

In [ ]:
bedrock_embeddings = create_vector_embedding_with_bedrock(docs[0].page_content, bedrock_client)
      
print(bedrock_embeddings)

## Storing embedding in Amazon OpenSearch serverless

Following the similar pattern embeddings could be generated for the entire corpus and stored in a vector store.

First of all we have to create a vector store. In this notebook we will use Amazon OpenSearch serverless.

Amazon OpenSearch Serverless is a serverless option in Amazon OpenSearch Service. As a developer, you can use OpenSearch Serverless to run petabyte-scale workloads without configuring, managing, and scaling OpenSearch clusters. You get the same interactive millisecond response times as OpenSearch Service with the simplicity of a serverless environment. Pay only for what you use by automatically scaling resources to provide the right amount of capacity for your application—without impacting data ingestion.

### Create a vector store - OpenSearch Serverless index

Before creating the new vector search collection and index, we must first create three associated OpenSearch policies: encryption security policy, network security policy, and data access policy.

In [ ]:
import random
suffix = random.randrange(200, 900)

identity = boto3.client('sts').get_caller_identity()['Arn']

def create_policies_in_oss(vector_store_name, aoss_client, role_arn):
    
    encryption_policy_name = f"cohere-sample-sp-{suffix}"
    network_policy_name = f"cohere-sample-np-{suffix}"
    access_policy_name = f'cohere-sample-ap-{suffix}'

    try:
        encryption_policy = aoss_client.create_security_policy(
            name=encryption_policy_name,
            policy=json.dumps(
                {
                    'Rules': [{'Resource': ['collection/' + vector_store_name],
                               'ResourceType': 'collection'}],
                    'AWSOwnedKey': True
                }),
            type='encryption'
        )
    except Exception as ex:
        print(ex)
    
    try:
        network_policy = aoss_client.create_security_policy(
            name=network_policy_name,
            policy=json.dumps(
                [
                    {'Rules': [{'Resource': ['collection/' + vector_store_name],
                                'ResourceType': 'collection'}],
                     'AllowFromPublic': True}
                ]),
            type='network'
        )
    except Exception as ex:
        print(ex)
    
    try:
        
        access_policy = aoss_client.create_access_policy(
            name=access_policy_name,
            policy=json.dumps(
                [
                    {
                        'Rules': [
                            {
                                'Resource': ['collection/' + vector_store_name],
                                'Permission': [
                                    'aoss:CreateCollectionItems',
                                    'aoss:DeleteCollectionItems',
                                    'aoss:UpdateCollectionItems',
                                    'aoss:DescribeCollectionItems'],
                                'ResourceType': 'collection'
                            },
                            {
                                'Resource': ['index/' + vector_store_name + '/*'],
                                'Permission': [
                                    'aoss:CreateIndex',
                                    'aoss:DeleteIndex',
                                    'aoss:UpdateIndex',
                                    'aoss:DescribeIndex',
                                    'aoss:ReadDocument',
                                    'aoss:WriteDocument'],
                                'ResourceType': 'index'
                            }],
                        'Principal': [identity, role_arn],
                        'Description': 'Easy data policy'}
                ]),
            type='data'
        )
    except Exception as ex:
        print(ex)
        
    return encryption_policy, network_policy, access_policy

### Create a new collection of type VECTORSEARCH

In [ ]:
# Create Collection
vector_store_name = f'cohere-embedding-collection-{suffix}'

encryption_policy, network_policy, access_policy = create_policies_in_oss(vector_store_name=vector_store_name,
                       aoss_client=aoss_client,
                       role_arn=sagemaker_role_arn)

In [ ]:
collection = aoss_client.create_collection(name=vector_store_name,type='VECTORSEARCH')

### Setting up the Amazon OpenSearch Serverless index using KNN settings
Once the OpenSearch collection is created, create an index to store the embeddings. The index settings must be configured beforehand to enable the KNN functionality using the following configuration:

In [ ]:
collection_id = collection['createCollectionDetail']['id']
host = collection_id + '.' + region_name + '.aoss.amazonaws.com'
print(host)

In [ ]:
service = 'aoss'
credentials = boto3.Session().get_credentials()
awsauth = AWSV4SignerAuth(credentials, region_name, service)

index_name = f"cohere-embedding-index"
index_body = {
   "settings": {
       "index":{
          "knn": "true",
       }
   },
   "mappings": {
      "properties": {
         "vector_field": {
            "type": "knn_vector",
            "dimension": 1536 
         },
          "text": {
                    "type": "keyword"
        }
      }
   }
}


In [ ]:
# Build the OpenSearch client
oss_client = OpenSearch(
    hosts=[{'host': host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)
# # It can take up to a minute for data access rules to be enforced
time.sleep(60)

To confirm its creation, we can retrieve the description of the new vector index you just created

In [ ]:
# We would get an index already exists exception if the index already exists, and that is fine.
try:
    response = oss_client.indices.create(index_name, body=index_body)
    print(f"response received for the create index -> {response}")
except Exception as e:
    print(f"error in creating index={index_name}, exception={e}")

Now we are ready to inject our documents into vector store. 

In [ ]:
# deleting indices
# aoss_client.indices.delete(index=index_name)

### Ingest the embeddings

Next you need to loop through your dataset and ingest items data into the cluster. A more robust and scalable solution for the embedding ingestion can be found in [Ingesting enriched data into Amazon ES](https://aws.amazon.com/blogs/industries/novartis-ag-uses-amazon-elasticsearch-k-nearest-neighbor-knn-and-amazon-sagemaker-to-power-search-and-recommendation/). 

In [ ]:
for idx in range(len(docs)): 
    embedding = create_vector_embedding_with_bedrock(docs[idx].page_content, bedrock_client)
    document = {
                'vector_field': embedding['vector_field'],
                'text': embedding['text']
                }
    response = oss_client.index(
    index = index_name,
    body = document
    )


## Perform Search based on Text Input

Let’s take a look at the results of a simple query. In below example, we'll receive an text input from user, and then will send it to search engine to get the relevant results.

In [ ]:
query_prompt = "Is it possible that I get sentenced to jail due to failure in filings?"
#query_prompt = "Who Must File Form 8300?"

In [ ]:
query_emb = create_vector_embedding_with_bedrock(query_prompt, bedrock_client)['vector_field']

In [ ]:
body = {
        "size": 5,
        "query": {
            "knn": {
                "vector_field": {
                    "vector": query_emb,
                    "k": 5,
                }
            }
        },
    }     

In [ ]:
res = oss_client.search(index=index_name, body=body)
results = ""
for hit in res["hits"]["hits"]:
    id_ = hit["_id"]
    text = hit["_source"]["text"]
    # results.append(text)
    results += text



## Generative Question Answering

Define utility function for conversation with Bedrock converse API

Model used - Cohere Command R+: A powerful Large Language Model (LLM) capable of understanding and generating text in multiple languages.

In [ ]:
def generate_conversation(
    bedrock_client,
    model_id,
    system_prompt,
    prompt,
    chat_history=[],
    temperature=0.3,
    max_tokens=400,
    top_p=0.95
):
    """
    Sends messages to a model.
    Args:
        bedrock_client: The Boto3 Bedrock runtime client.
        model_id (str): The model ID to use.
        system_prompt (str) : The system prompt for the model to use.
        prompt (str) : The message/question to send to the model.
        chat_history (list): The chat history from user and assistant.

    Returns:
        response (str): The text generated output from the model.
        chat_history (str): The full conversation between user and assistant that the model generated.

    """

    system_prompts = [
        {
            "text": system_prompt
        }
    ]

    messages = [
        {
            "role": "user",
            "content": [{"text": prompt}]
        }
    ]

    chat_history.extend(messages)

    # Base inference parameters.
    inference_config = {
        "temperature": temperature,
        "maxTokens": max_tokens,
        "topP": top_p,
    }

    # Additional inference parameters to use.
    additional_model_fields = {}

    # Send the message.
    response = bedrock_client.converse(
        modelId=model_id,
        messages=messages,
        system=system_prompts,
        inferenceConfig=inference_config,
        additionalModelRequestFields=additional_model_fields
    )

    chat_history.append(response["output"]["message"])

    return response["output"]["message"]["content"][0]["text"], chat_history

Define the system prompt and guardrails

In [ ]:
system_prompt = """You are an AI assistant. Your knowledge is based solely on the information provided between the <documents> and </documents> tags.

Before answering any questions, first check if the user has provided information between the <documents> and </documents> tags. If no information is provided, respond with the following JSON:

{
    "answer": "I do not have enough information to answer that question."
}

If documents are provided, your task is to answer questions accurately and concisely, using only the details from the given documents. Do not use your own knowledge or any external sources to answer the questions, even if you know the answer.

If a question cannot be fully answered using the provided documents, respond with the following JSON:

{
    "answer": "I do not have enough information to answer that question."
}

All responses must be in valid JSON format, with the 'answer' key containing the actual response text.

To provide transparency, include your reasoning process with the 'thinking' key as the following format:

{
    "answer": "Your response here",
    "thinking": "Your reasoning process here"
}

Be concise and objective in your responses, without any personal opinions or subjective statements.
"""
prompt_template = "<documents>\n{documents}\n</documents>\n\nQuestion: {question}\nThink step-by-step."

In [ ]:
# Define model ID parameter
model_id = "cohere.command-r-plus-v1:0"

In [ ]:
chat_history = []
prompt = prompt_template.format(documents=results, question=query_prompt)
response, chat_history = generate_conversation(
    bedrock_client,
    model_id,
    system_prompt,
    prompt,
    chat_history
)
print(response)

## 6. Clean up

When you finish this exercise, remove your resources with the following steps:

Delete vector index.
Delete data, network, and encryption access ploicies.
Delete collection.
Delete SageMaker Studio user profile and domain.
Optionally, empty and delete the S3 bucket, or keep whatever you want.  

In [ ]:
# delete vector index
oss_client.indices.delete(index=index_name)

# delete data, network, and encryption access ploicies
aoss_client.delete_access_policy(type="data", name=access_policy['accessPolicyDetail']['name'])
aoss_client.delete_security_policy(type="network", name=network_policy['securityPolicyDetail']['name'])
aoss_client.delete_security_policy(type="encryption", name=encryption_policy['securityPolicyDetail']['name'])

# delete collection
collection_id = collection['createCollectionDetail']['id']
aoss_client.delete_collection(id=collection_id)